# Football Match Predictor - Feature Engineering

## Library Imports

In [1]:
# Add all the library imports required
import pandas as pd
import numpy as np
import pickle
import data_cleaning
from tqdm import tqdm
import os

## Download the ELO Data

In [2]:
if not os.path.exists('./elo_dict.pkl'):
    !wget "https://aicore-files.s3.amazonaws.com/Data-Science/elo_dict.pkl"

In [3]:
elo_dict = pickle.load(open('elo_dict.pkl', 'rb'))
print(list(elo_dict.keys())[0])
print(elo_dict[list(elo_dict.keys())[0]])

https://www.besoccer.com/match/saarbrucken/stuttgarter-kickers/19903487
{'Elo_home': 56.0, 'Elo_away': 59.0}


In [4]:
elo_link_list = []
elo_home_list = []
elo_away_list = []
for key, value in elo_dict.items():
    elo_link_list.append(key)
    elo_home_list.append(value['Elo_home'])
    elo_away_list.append(value['Elo_away'])

elo_df = pd.DataFrame({'link': elo_link_list, 'home_elo': elo_home_list, 'away_elo': elo_away_list})
elo_df.head()

,link,home_elo,away_elo
0,https://www.besoccer.com/match/saarbrucken/stu...,56.0,59.0
1,https://www.besoccer.com/match/sc-freiburg/unt...,53.0,55.0
2,https://www.besoccer.com/match/vfl-osnabruck/m...,52.0,53.0
3,https://www.besoccer.com/match/rot-weiss-essen...,53.0,62.0
4,https://www.besoccer.com/match/alemannia-aache...,57.0,52.0


## Download the other data and join into one dataframe

In [5]:
scores_df = data_cleaning.import_leagues()
scores_df = data_cleaning.tweak_scores_df(scores_df)
scores_df = data_cleaning.create_match_id_col_from_link(scores_df)
scores_df.head()

,home_team,away_team,score,link,season_year,match_round,league,home_goals,away_goals,result,home_points,away_points,match_id
0,Charlton Athletic,Derby County,0-0,https://www.besoccer.com/match/charlton-athlet...,1990,1,premier_league,0,0,draw,1,1,charlton-athletic-fc/derby-county-fc/1990
1,Tottenham Hotspur,Luton Town,2-1,https://www.besoccer.com/match/tottenham-hotsp...,1990,1,premier_league,2,1,home_win,3,0,tottenham-hotspur-fc/luton-town-fc/1990
2,Southampton,Millwall,1-2,https://www.besoccer.com/match/southampton-fc/...,1990,1,premier_league,1,2,away_win,0,3,southampton-fc/millwall-fc/1990
3,Sheffield Wednesday,Norwich City,0-2,https://www.besoccer.com/match/sheffield-wedne...,1990,1,premier_league,0,2,away_win,0,3,sheffield-wednesday-fc/norwich-city-fc/1990
4,Queens Park Rangers,Crystal Palace,2-0,https://www.besoccer.com/match/queens-park-ran...,1990,1,premier_league,2,0,home_win,3,0,queens-park-rangers-fc/crystal-palace-fc/1990


In [6]:
scores_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122619 entries, 0 to 129062
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   home_team    122619 non-null  object  
 1   away_team    122619 non-null  object  
 2   score        122619 non-null  object  
 3   link         122619 non-null  object  
 4   season_year  122619 non-null  int64   
 5   match_round  122619 non-null  int64   
 6   league       122619 non-null  object  
 7   home_goals   122619 non-null  int64   
 8   away_goals   122619 non-null  int64   
 9   result       122619 non-null  category
 10  home_points  122619 non-null  int64   
 11  away_points  122619 non-null  int64   
 12  match_id     122619 non-null  object  
dtypes: category(1), int64(6), object(6)
memory usage: 12.3+ MB


In [7]:
scores_df.match_id.nunique()

122619

In [8]:
match_info_df = data_cleaning.import_match_info_data()
match_info_df = data_cleaning.create_match_id_col(match_info_df)
match_info_df.head()

,date,referee,home_yellow,home_red,away_yellow,away_red,match_id
0,1989-07-29 15:00:00,Hans-Jürgen Weber,0.0,0.0,3.0,0.0,saarbrucken/stuttgarter-kickers/1990
1,1989-07-29 15:00:00,Kurt Wittke,1.0,0.0,0.0,0.0,sc-freiburg/unterhaching/1990
2,1989-07-29 15:00:00,Werner Föckler,3.0,0.0,2.0,0.0,vfl-osnabruck/meppen/1990
3,1989-07-29 15:00:00,Heinz Werner,2.0,0.0,2.0,0.0,rot-weiss-essen/schalke-04/1990
4,1989-07-29 15:00:00,Hans-Peter Dellwing,1.0,0.0,1.0,0.0,alemannia-aachen/msv-duisburg/1990


In [9]:
match_info_df.dtypes

date           datetime64[ns]
referee                object
home_yellow           float64
home_red              float64
away_yellow           float64
away_red              float64
match_id               object
dtype: object

In [10]:
match_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143348 entries, 0 to 143347
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         143348 non-null  datetime64[ns]
 1   referee      113721 non-null  object        
 2   home_yellow  122798 non-null  float64       
 3   home_red     122798 non-null  float64       
 4   away_yellow  122798 non-null  float64       
 5   away_red     122798 non-null  float64       
 6   match_id     143348 non-null  object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 7.7+ MB


In [11]:
match_info_df.match_id.nunique()

143348

In [12]:
team_info_df = data_cleaning.import_team_info_data()
team_info_df.head()

,team,city,country,stadium,capacity,pitch
0,Wattenscheid 09,Bochum-Wattenscheid,Germany,Lohrheidestadion,16233,Natural
1,Hertha BSC,Berlín,Germany,Olympiastadion Berlin,76065,Natural
2,Unterhaching,Unterhaching,Germany,Sportpark Unterhaching,15053,Natural
3,Fortuna Köln,Cologne,Germany,Südstadion,14944,Natural
4,MSV Duisburg,Duisburgo,Germany,Schauinsland-Reisen-Arena,31514,Natural


In [13]:
scores_match_info_df = pd.merge(scores_df, match_info_df, how='left', on="match_id")
scores_match_info_df.head()

,home_team,away_team,score,link,season_year,match_round,league,home_goals,away_goals,result,home_points,away_points,match_id,date,referee,home_yellow,home_red,away_yellow,away_red
0,Charlton Athletic,Derby County,0-0,https://www.besoccer.com/match/charlton-athlet...,1990,1,premier_league,0,0,draw,1,1,charlton-athletic-fc/derby-county-fc/1990,1989-08-19,NaN,0.0,0.0,0.0,0.0
1,Tottenham Hotspur,Luton Town,2-1,https://www.besoccer.com/match/tottenham-hotsp...,1990,1,premier_league,2,1,home_win,3,0,tottenham-hotspur-fc/luton-town-fc/1990,1989-08-19,NaN,0.0,0.0,0.0,0.0
2,Southampton,Millwall,1-2,https://www.besoccer.com/match/southampton-fc/...,1990,1,premier_league,1,2,away_win,0,3,southampton-fc/millwall-fc/1990,1989-08-19,NaN,0.0,0.0,0.0,0.0
3,Sheffield Wednesday,Norwich City,0-2,https://www.besoccer.com/match/sheffield-wedne...,1990,1,premier_league,0,2,away_win,0,3,sheffield-wednesday-fc/norwich-city-fc/1990,1989-08-19,NaN,0.0,0.0,0.0,0.0
4,Queens Park Rangers,Crystal Palace,2-0,https://www.besoccer.com/match/queens-park-ran...,1990,1,premier_league,2,0,home_win,3,0,queens-park-rangers-fc/crystal-palace-fc/1990,1989-08-19,NaN,0.0,0.0,0.0,0.0


In [14]:
scores_match_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122619 entries, 0 to 122618
Data columns (total 19 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   home_team    122619 non-null  object        
 1   away_team    122619 non-null  object        
 2   score        122619 non-null  object        
 3   link         122619 non-null  object        
 4   season_year  122619 non-null  int64         
 5   match_round  122619 non-null  int64         
 6   league       122619 non-null  object        
 7   home_goals   122619 non-null  int64         
 8   away_goals   122619 non-null  int64         
 9   result       122619 non-null  category      
 10  home_points  122619 non-null  int64         
 11  away_points  122619 non-null  int64         
 12  match_id     122619 non-null  object        
 13  date         119163 non-null  datetime64[ns]
 14  referee      94378 non-null   object        
 15  home_yellow  102511 non-null  floa

In [15]:
scores_match_info_df[scores_match_info_df.date.isna()]

,home_team,away_team,score,link,season_year,match_round,league,home_goals,away_goals,result,home_points,away_points,match_id,date,referee,home_yellow,home_red,away_yellow,away_red
12287,Aston Villa,Tottenham Hotspur,0-2,https://www.besoccer.com/match/aston-villa-fc/...,2021,18,premier_league,0,2,away_win,0,3,aston-villa-fc/tottenham-hotspur-fc/2021,NaT,NaN,NaN,NaN,NaN,NaN
12293,Aston Villa,Everton,0-0,https://www.besoccer.com/match/aston-villa-fc/...,2021,19,premier_league,0,0,draw,1,1,aston-villa-fc/everton-fc/2021,NaT,NaN,NaN,NaN,NaN,NaN
12367,Everton,Southampton,1-0,https://www.besoccer.com/match/everton-fc/sout...,2021,26,premier_league,1,0,home_win,3,0,everton-fc/southampton-fc/2021,NaT,NaN,NaN,NaN,NaN,NaN
12368,Burnley,Arsenal,1-1,https://www.besoccer.com/match/burnley-fc/arse...,2021,27,premier_league,1,1,draw,1,1,burnley-fc/arsenal/2021,NaT,NaN,NaN,NaN,NaN,NaN
12369,Sheffield United,Southampton,0-2,https://www.besoccer.com/match/sheffield-unite...,2021,27,premier_league,0,2,away_win,0,3,sheffield-united/southampton-fc/2021,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122428,PEC Zwolle,Twente 1965,1-0,https://www.besoccer.com/match/fc-zwolle/fc-tw...,2021,29,eredivisie,1,0,home_win,3,0,fc-zwolle/fc-twente-1965/2021,NaT,NaN,NaN,NaN,NaN,NaN
122429,Utrecht,Feyenoord,1-2,https://www.besoccer.com/match/fc-utrecht/feye...,2021,29,eredivisie,1,2,away_win,0,3,fc-utrecht/feyenoord/2021,NaT,NaN,NaN,NaN,NaN,NaN
122430,Groningen,Heerenveen,0-2,https://www.besoccer.com/match/fc-groningen/he...,2021,29,eredivisie,0,2,away_win,0,3,fc-groningen/heerenveen/2021,NaT,NaN,NaN,NaN,NaN,NaN
122431,VVV Venlo,PSV,0-2,https://www.besoccer.com/match/vvv/psv/202113372,2021,29,eredivisie,0,2,away_win,0,3,vvv/psv/2021,NaT,NaN,NaN,NaN,NaN,NaN


In [16]:
team_info_reduced_df = team_info_df.copy()
team_info_reduced_df = team_info_reduced_df[['team', 'capacity']]

scores_match_team_info_df = (pd.merge(scores_match_info_df, team_info_reduced_df, how='left', left_on='home_team', right_on='team')
                                .drop(columns = 'team'))
 
scores_match_team_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122619 entries, 0 to 122618
Data columns (total 20 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   home_team    122619 non-null  object        
 1   away_team    122619 non-null  object        
 2   score        122619 non-null  object        
 3   link         122619 non-null  object        
 4   season_year  122619 non-null  int64         
 5   match_round  122619 non-null  int64         
 6   league       122619 non-null  object        
 7   home_goals   122619 non-null  int64         
 8   away_goals   122619 non-null  int64         
 9   result       122619 non-null  category      
 10  home_points  122619 non-null  int64         
 11  away_points  122619 non-null  int64         
 12  match_id     122619 non-null  object        
 13  date         119163 non-null  datetime64[ns]
 14  referee      94378 non-null   object        
 15  home_yellow  102511 non-null  floa

In [17]:
capcity_na_idxs = scores_match_team_info_df['capacity'].isna().to_list()
scores_match_team_info_df.loc[capcity_na_idxs, 'home_team'].unique()

array(['Sheffield Wednesday', 'Queens Park Rangers',
       'Oldham Athletic AFC', 'West Bromwich Albion',
       'Brighton & Hove Albion', 'Peterborough United',
       'Gimnàstic Tarragona', 'Real Unión de Irún', 'Fortuna Düsseldorf',
       'Eintracht Frankfurt', 'B. Mönchengladbach', 'Stuttgarter Kickers',
       '1. FC Lokomotive Leipzig', 'Blau-Weiß 1890 Berlin',
       'FC Carl Zeiss Jena', 'Rot-Weiß Oberhausen',
       'Kickers Offenbach FC', 'SV Eintracht Trier',
       'SV Wacker Burghausen', 'Siegen Sportfreunde ',
       'SV Wehen Burghausen', 'Würzburger Kickers', 'Barletta', 'Licata',
       'AS Lucchese Libertas 1905', 'Taranto', 'Casertana',
       'Calcio Portogruaro-Summaga', 'Sporting Toulon Var',
       'Olympique Marseille', 'Evian Thonon Gaillard',
       'CS Louhans Cuiseaux', 'FC Libourne Saint Seurin'], dtype=object)

In [18]:
scores_match_team_info_elo_df = pd.merge(scores_match_team_info_df, elo_df, how='left', on="link").set_index('match_id')
# scores_match_team_info_elo_df.dropna(subset=['score'], inplace=True) 
scores_match_team_info_elo_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 122619 entries, charlton-athletic-fc/derby-county-fc/1990 to graafschap/fc-den-bosch/2021
Data columns (total 21 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   home_team    122619 non-null  object        
 1   away_team    122619 non-null  object        
 2   score        122619 non-null  object        
 3   link         122619 non-null  object        
 4   season_year  122619 non-null  int64         
 5   match_round  122619 non-null  int64         
 6   league       122619 non-null  object        
 7   home_goals   122619 non-null  int64         
 8   away_goals   122619 non-null  int64         
 9   result       122619 non-null  category      
 10  home_points  122619 non-null  int64         
 11  away_points  122619 non-null  int64         
 12  date         119163 non-null  datetime64[ns]
 13  referee      94378 non-null   object        
 14  home_yellow  102511 non-nul

In [19]:
elo_na_idxs = scores_match_team_info_elo_df['home_elo'].isna().to_list()
scores_match_team_info_elo_df.loc[elo_na_idxs]

,home_team,away_team,score,link,season_year,match_round,league,home_goals,away_goals,result,...,away_points,date,referee,home_yellow,home_red,away_yellow,away_red,capacity,home_elo,away_elo
match_id,,,,,,,,,,,,,,,,,,,,,
sheffield-wednesday-fc/norwich-city-fc/1993,Sheffield Wednesday,Norwich City,1-0,https://www.besoccer.com/match/sheffield-wedne...,1993,23,premier_league,1,0,home_win,...,0,1993-01-10 00:00:00,David Allison,0.0,0.0,0.0,0.0,NaN,NaN,NaN
aston-villa-fc/arsenal/2006,Aston Villa,Arsenal,0-0,https://www.besoccer.com/match/aston-villa-fc/...,2006,20,premier_league,0,0,draw,...,1,2005-12-31 14:45:00,Uriah Rennie,1.0,0.0,1.0,0.0,42788.0,NaN,NaN
charlton-athletic-fc/west-ham-united/2006,Charlton Athletic,West Ham,2-0,https://www.besoccer.com/match/charlton-athlet...,2006,20,premier_league,2,0,home_win,...,0,2005-12-31 15:00:00,Graham Poll,1.0,0.0,0.0,0.0,27111.0,NaN,NaN
chelsea-fc/birmingham-city-fc/2006,Chelsea,Birmingham City,2-0,https://www.besoccer.com/match/chelsea-fc/birm...,2006,20,premier_league,2,0,home_win,...,0,2005-12-31 15:00:00,Mike Dean,1.0,0.0,0.0,0.0,41841.0,NaN,NaN
tottenham-hotspur-fc/newcastle-united-fc/2006,Tottenham Hotspur,Newcastle,2-0,https://www.besoccer.com/match/tottenham-hotsp...,2006,20,premier_league,2,0,home_win,...,0,2005-12-31 15:00:00,Howard Webb,0.0,0.0,1.0,0.0,62062.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mvv/agovv-apeldoorn/2006,MVV Maastricht,AGOVV Apeldoorn,2-2,https://www.besoccer.com/match/mvv/agovv-apeld...,2006,1,eerste_divisie,2,2,draw,...,1,2005-08-12 20:00:00,Bas de Groot,0.0,0.0,0.0,0.0,10000.0,NaN,NaN
emmen/fc-den-bosch/2006,Emmen,Den Bosch,3-1,https://www.besoccer.com/match/emmen/fc-den-bo...,2006,1,eerste_divisie,3,1,home_win,...,0,2005-08-12 20:00:00,Ed Janssen,1.0,1.0,2.0,0.0,8600.0,NaN,NaN
vvv/stormvogels-telstar/2006,VVV Venlo,SC Telstar,1-0,https://www.besoccer.com/match/vvv/stormvogels...,2006,1,eerste_divisie,1,0,home_win,...,0,2005-08-12 20:00:00,Jack van Hulten,1.0,0.0,4.0,0.0,8000.0,NaN,NaN


In [20]:
# Check for duplicates
scores_match_team_info_elo_df.duplicated(subset=['home_team', 'away_team', 'season_year'], keep=False).sum()

0

In [21]:
scores_match_team_info_elo_df.shape

(122619, 21)

In [22]:
scores_match_team_info_elo_df = (scores_match_team_info_elo_df
    .assign(missing_elo = scores_match_team_info_elo_df.home_elo.isna(),
        missing_cards = scores_match_team_info_elo_df.home_yellow.isna()))

## Clean the missing data

In [23]:
scores_match_team_info_elo_df.isna().sum()[scores_match_team_info_elo_df.isna().sum() > 0]

date            3456
referee        28241
home_yellow    20108
home_red       20108
away_yellow    20108
away_red       20108
capacity        5507
home_elo        9469
away_elo        9469
dtype: int64

What to do with each column with missing data:
- **Date**: Nothing, drop column (for now anyway)
- **Referee**: Nothing, drop column (for now anyway)
- **Home/Away yellow/red**: look at season data, if partially na, use ffill (groupby team and season)
- **Capacity**: use some form of imputation, maybe kNN or median
- **Elo home/away**: use some form of imputation, maybe kNN or median, however, it may be the less good teams that don't have an elo

### Cards

In [24]:
missing_cards_df = pd.concat([(scores_match_team_info_elo_df
    .assign(yellow_missing = scores_match_team_info_elo_df.home_yellow.isna())
    .groupby(['season_year', 'league'])
    .agg(missing_cards = ('yellow_missing', 'sum'))
), (scores_match_team_info_elo_df
    .groupby(['season_year', 'league'])
    .agg(total_games = ('season_year', 'count'))
)], axis=1)

# so there only certain seasons where the full season has missing cards data
year_season_full_missing_cards = (missing_cards_df[missing_cards_df.missing_cards > 0]
    [(missing_cards_df[missing_cards_df.missing_cards > 0].missing_cards) == (missing_cards_df[missing_cards_df.missing_cards > 0].total_games)]
).index.to_list()

# Going to calculate the median value for each year and replace the missing season value with that value
years_missing, leagues_missing = tuple(map(list, zip(*year_season_full_missing_cards)))
median_values_for_missing_years_df = pd.DataFrame()
for year in set(years_missing):
    median_values_for_missing_years_df[year] = (scores_match_team_info_elo_df
        .query("season_year == @year")
        [['home_yellow', 'home_red', 'away_yellow', 'away_red']]
        .median()
    )


median_values_for_missing_years_df

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2004,2006,2007,2011
home_yellow,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
home_red,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
away_yellow,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
away_red,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
for year, league in year_season_full_missing_cards:
    
    selection_idx = (scores_match_team_info_elo_df.season_year == year) & (scores_match_team_info_elo_df.league == league)

    scores_match_team_info_elo_df.loc[selection_idx, ['home_yellow', 'home_red', 'away_yellow', 'away_red']] = (scores_match_team_info_elo_df
        .query("(season_year == @year) and (league == @league)")
        [['home_yellow', 'home_red', 'away_yellow', 'away_red']]
        .fillna(median_values_for_missing_years_df.to_dict()[year])
    )


In [26]:
missing_cards_df = pd.concat([(scores_match_team_info_elo_df
    .assign(yellow_missing = scores_match_team_info_elo_df.home_yellow.isna())
    .groupby(['season_year', 'league'])
    .agg(missing_cards = ('yellow_missing', 'sum'))
), (scores_match_team_info_elo_df
    .groupby(['season_year', 'league'])
    .agg(total_games = ('season_year', 'count'))
)], axis=1)

missing_cards_df

missing_cards  total_games
season_year league                                      
1990        2_liga                        0          380
            bundesliga                    0          306
            championship                  0          552
            eredivisie                    0          306
            ligue_1                       0           10
...                                     ...          ...
2021        premier_league               52          309
            primera_division             53          300
            segunda_division             76          372
            serie_a                      61          299
            serie_b                      79          328

[348 rows x 2 columns]

In [27]:
missing_cards_teams_df = (((pd.concat([(scores_match_team_info_elo_df
            [['season_year', 'league', 'home_team', 'home_yellow']]
            .rename(columns = lambda col_name: col_name[5:] if col_name[:4] == 'home' else col_name)), 
            (scores_match_team_info_elo_df
            [['season_year', 'league', 'away_team', 'away_yellow']]
            .rename(columns = lambda col_name: col_name[5:] if col_name[:4] == 'away' else col_name))], axis=0))
        .assign(idx = lambda df_: np.arange(df_.shape[0]))
        .set_index('idx'))
    .assign(missing_yellow = lambda df_: df_.yellow.isna())
    .groupby(['season_year', 'league', 'team'])
    .agg(missing_cards=('missing_yellow', 'sum'),
        total_games = ('missing_yellow', 'count'))
)
# 
# so there certain seasons where some teams have no cards data for the full season
missing_cards_teams_full_season_idx = missing_cards_teams_df[missing_cards_teams_df.missing_cards == missing_cards_teams_df.total_games].index.to_list()
len(missing_cards_teams_full_season_idx)

208

In [28]:
year, league, _ = tuple(zip(*missing_cards_teams_full_season_idx))
unique_year_league_set = list(set(zip(year, league)))
cards_league_year = dict()

for year_league in unique_year_league_set:
    year, league = year_league
    cards_league_year[year_league] = (scores_match_team_info_elo_df
        .query("(season_year == @year) and (league == @league)")
        [['home_yellow', 'home_red', 'away_yellow', 'away_red']]
        .median()
        .to_dict()
    )
cards_league_year[list(cards_league_year.keys())[0]]

{'home_yellow': 2.5, 'home_red': 0.0, 'away_yellow': 4.0, 'away_red': 0.0}

In [29]:
for year_league_team in missing_cards_teams_full_season_idx:
    year, league, team = year_league_team
    year_league = (year, league)
    home_selection_idx = ((scores_match_team_info_elo_df.season_year == year) & (scores_match_team_info_elo_df.league == league)
        & (scores_match_team_info_elo_df.home_team == team))
    away_selection_idx = ((scores_match_team_info_elo_df.season_year == year) & (scores_match_team_info_elo_df.league == league)
        & (scores_match_team_info_elo_df.away_team == team))
    
    home_dict = {'home_yellow': cards_league_year[year_league]['home_yellow'], 
        'home_red': cards_league_year[year_league]['home_red']}
    away_dict = {'away_yellow': cards_league_year[year_league]['away_yellow'], 
        'away_red': cards_league_year[year_league]['away_red']}
    
    scores_match_team_info_elo_df.loc[home_selection_idx, ['home_yellow', 'home_red']] = (scores_match_team_info_elo_df
        .query("(season_year == @year) and (league == @league) and (home_team == @team)")
        [['home_yellow', 'home_red']]
        .fillna(home_dict))
    
    scores_match_team_info_elo_df.loc[away_selection_idx, ['away_yellow', 'away_red']] = (scores_match_team_info_elo_df
        .query("(season_year == @year) and (league == @league) and (away_team == @team)")
        [['away_yellow', 'away_red']]
        .fillna(away_dict))


In [30]:
missing_cards_df2 = pd.concat([(scores_match_team_info_elo_df
    .assign(yellow_missing = scores_match_team_info_elo_df.home_yellow.isna())
    .groupby(['season_year', 'league'])
    .agg(missing_cards = ('yellow_missing', 'sum'))
), (scores_match_team_info_elo_df
    .groupby(['season_year', 'league'])
    .agg(total_games = ('season_year', 'count'))
)], axis=1)

# so there only certain seasons where the full season has missing cards data
(missing_cards_df2[missing_cards_df2.missing_cards > 0]
    [(missing_cards_df2[missing_cards_df2.missing_cards > 0].missing_cards) == (missing_cards_df2[missing_cards_df2.missing_cards > 0].total_games)]
)

,,missing_cards,total_games
season_year,league,,


The rest of the missing cards will be filled in later on using ffill

### Capacity

In [31]:
(pd.concat([(scores_match_team_info_elo_df
    .assign(missing_capacity = scores_match_team_info_elo_df.capacity.isna())
    .groupby(['season_year', 'league'])
    .missing_capacity
    .sum()), (scores_match_team_info_elo_df
        .groupby(['season_year', 'league'])
        .agg(num_matches = ('match_round', 'count')))]
    , axis=1)
)

missing_capacity  num_matches
season_year league                                         
1990        2_liga                          38          380
            bundesliga                      51          306
            championship                    69          552
            eredivisie                       0          306
            ligue_1                          1           10
...                                        ...          ...
2021        premier_league                  32          309
            primera_division                 0          300
            segunda_division                 0          372
            serie_a                          0          299
            serie_b                          0          328

[348 rows x 2 columns]

In [32]:
missing_capcity_df = pd.concat([(scores_match_team_info_elo_df
    .assign(missing_capacity = scores_match_team_info_elo_df.capacity.isna())
    .groupby(['season_year', 'league'])
    .missing_capacity
    .sum()
), (scores_match_team_info_elo_df
    .groupby(['season_year', 'league'])
    .agg(total_games = ('season_year', 'count'))
)], axis=1)

# it is different for capacity because we want to fill in all missing data with a new value
seasons_with_some_missing_capacity = missing_capcity_df[missing_capcity_df.missing_capacity > 0].index.to_list()

median_values_for_missing_capacity = {}
for year_league in seasons_with_some_missing_capacity:
    year, league = year_league
    median_values_for_missing_capacity[year_league] = (scores_match_team_info_elo_df
        .query("(season_year == @year) and (league == @league)")
        .capacity
        .median()
    )

median_values_for_missing_capacity



{(1990, '2_liga'): 21075.0,
 (1990, 'bundesliga'): 42358.0,
 (1990, 'championship'): 28383.0,
 (1990, 'ligue_1'): 35472.0,
 (1990, 'premier_league'): 33143.0,
 (1990, 'serie_b'): 24530.5,
 (1991, '2_liga'): 19350.0,
 (1991, 'bundesliga'): 49780.0,
 (1991, 'championship'): 22253.0,
 (1991, 'ligue_1'): 32038.5,
 (1991, 'premier_league'): 40204.0,
 (1991, 'serie_b'): 25085.0,
 (1992, '2_liga'): 21719.5,
 (1992, 'bundesliga'): 46069.0,
 (1992, 'championship'): 20821.0,
 (1992, 'ligue_1'): 28894.0,
 (1992, 'premier_league'): 40204.0,
 (1992, 'serie_b'): 24026.0,
 (1993, '2_liga'): 23196.0,
 (1993, 'bundesliga'): 46069.0,
 (1993, 'championship'): 20146.0,
 (1993, 'ligue_1'): 31090.0,
 (1993, 'premier_league'): 34988.0,
 (1993, 'serie_b'): 20920.0,
 (1994, '2_liga'): 29273.0,
 (1994, 'bundesliga'): 49780.0,
 (1994, 'championship'): 24659.0,
 (1994, 'ligue_1'): 29230.0,
 (1994, 'premier_league'): 40569.0,
 (1994, 'serie_b'): 20987.0,
 (1995, '2_liga'): 27333.5,
 (1995, 'bundesliga'): 46069.0,


In [33]:
for key, capacity in median_values_for_missing_capacity.items():
    
    year, league = key
    
    selection_idx = (scores_match_team_info_elo_df.season_year == year) & (scores_match_team_info_elo_df.league == league)

    scores_match_team_info_elo_df.loc[selection_idx, 'capacity'] = (scores_match_team_info_elo_df
        .query("(season_year == @year) and (league == @league)")
        .capacity
        .fillna(capacity)
    )

In [34]:
scores_match_team_info_elo_df.isna().sum()[scores_match_team_info_elo_df.isna().sum() > 0]

date            3456
referee        28241
home_yellow     2854
home_red        2854
away_yellow     2854
away_red        2854
home_elo        9469
away_elo        9469
dtype: int64

### Elo home/away

In [35]:
(scores_match_team_info_elo_df
    .assign(elo_home_or_away_missing = (scores_match_team_info_elo_df.home_elo.isna()) & (scores_match_team_info_elo_df.away_elo.isna()),
        elo_home_missing = scores_match_team_info_elo_df.home_elo.isna(),
        elo_away_missing = scores_match_team_info_elo_df.away_elo.isna())
    .groupby(['season_year', 'league'])
    .agg(elo_home_or_away_missing = ('elo_home_or_away_missing', 'sum'),
        elo_home_missing = ('elo_home_missing', 'sum'),
        elo_away_missing = ('elo_away_missing', 'sum'),
        total_matches = ('season_year', 'count'))
)[(scores_match_team_info_elo_df
    .assign(elo_home_or_away_missing = (scores_match_team_info_elo_df.home_elo.isna()) & (scores_match_team_info_elo_df.away_elo.isna()))
    .groupby(['season_year', 'league'])
    .elo_home_or_away_missing
    .sum() > 0
)]

elo_home_or_away_missing  elo_home_missing  \
season_year league                                                         
1990        serie_b                                380               380   
1991        serie_b                                380               380   
1992        serie_a                                  6                 6   
            serie_b                                380               380   
1993        premier_league                           1                 1   
            primera_division                         1                 1   
            serie_b                                380               380   
1994        ligue_2                                462               462   
            serie_b                                380               380   
1995        championship                           552               552   
            ligue_2                                462               462   
1996        championship                           552               552   
            ligue_2                                  6                 6   
1997        championship                           552               552   
            ligue_2                                462               462   
            segunda_division                         1                 1   
1998        championship                           552               552   
            ligue_2                                462               462   
            serie_b                                380               380   
1999        championship                           552               552   
            eerste_divisie                           9                 9   
            serie_b                                380               380   
2000        eerste_divisie                           9                 9   
            serie_b                                380               380   
2001        championship                           552               552   
2002        serie_b                                  2                 2   
2004        serie_b                                 32                32   
2005        2_liga                                 306               306   
            championship                           552               552   
            eerste_divisie                           9                 9   
            eredivisie                             306               306   
2006        eerste_divisie                          10                10   
            premier_league                          10                10   
2012        premier_league                           7                 7   
2016        primera_division                         1                 1   
2021        premier_league                           1                 1   

                              elo_away_missing  total_matches  
season_year league                                             
1990        serie_b                        380            380  
1991        serie_b                        380            380  
1992        serie_a                          6            306  
            serie_b                        380            380  
1993        premier_league                   1            462  
            primera_division                 1            380  
            serie_b                        380            380  
1994        ligue_2                        462            462  
            serie_b                        380            380  
1995        championship                   552            552  
            ligue_2                        462            462  
1996        championship                   552            552  
            ligue_2                          6            462  
1997        championship                   552            552  
            ligue_2                        462            462  
            segunda_division                 1       

In [36]:
# Separate the seasons with some missing Elo data and with the full season missing the elo data

missing_elo_df = (scores_match_team_info_elo_df
    .assign(missing_elo_data = scores_match_team_info_elo_df.home_elo.isna())
    .groupby(['season_year', 'league'])
    .agg(missing_elo_data = ('missing_elo_data', 'sum'),
        total_matches = ('season_year', 'count'))
)[(scores_match_team_info_elo_df
    .assign(missing_elo_data = scores_match_team_info_elo_df.home_elo.isna())
    .groupby(['season_year', 'league'])
    .missing_elo_data
    .sum() > 0
)]

missing_elo_full_season_idx = missing_elo_df[missing_elo_df.missing_elo_data == missing_elo_df.total_matches].index.to_list()
missing_elo_partial_season_idx = missing_elo_df[missing_elo_df.missing_elo_data != missing_elo_df.total_matches].index.to_list()



In [37]:
# Use median from all games in the current year

(scores_match_team_info_elo_df
    .groupby('season_year')
    .agg(home_elo_mean = ('home_elo', 'mean'), away_elo_mean = ('away_elo', 'mean'),
        home_elo_median = ('home_elo', 'median'), away_elo_median = ('away_elo', 'median')))

,home_elo_mean,away_elo_mean,home_elo_median,away_elo_median
season_year,,,,
1990,68.600667,68.661333,69.0,69.0
1991,69.402967,69.447181,72.0,72.0
1992,70.979218,70.996638,73.0,73.0
1993,69.015461,69.057427,71.0,71.0
1994,69.663094,69.701333,72.0,72.0
1995,71.063631,71.103244,73.0,73.0
1996,69.271983,69.305025,70.0,70.0
1997,70.465775,70.504667,72.0,72.0
1998,71.636726,71.673669,74.0,74.0


In [38]:
median_values_for_full_season_missing_elo = {}
for year_league in missing_elo_full_season_idx:
    year, league = year_league
    median_values_for_full_season_missing_elo[year_league] = (scores_match_team_info_elo_df
        .query("(season_year == @year)")
        .loc[:,['home_elo', 'away_elo']]
        .median()
        .to_dict()
    )
median_values_for_full_season_missing_elo

{(1990, 'serie_b'): {'home_elo': 69.0, 'away_elo': 69.0},
 (1991, 'serie_b'): {'home_elo': 72.0, 'away_elo': 72.0},
 (1992, 'serie_b'): {'home_elo': 73.0, 'away_elo': 73.0},
 (1993, 'serie_b'): {'home_elo': 71.0, 'away_elo': 71.0},
 (1994, 'ligue_2'): {'home_elo': 72.0, 'away_elo': 72.0},
 (1994, 'serie_b'): {'home_elo': 72.0, 'away_elo': 72.0},
 (1995, 'championship'): {'home_elo': 73.0, 'away_elo': 73.0},
 (1995, 'ligue_2'): {'home_elo': 73.0, 'away_elo': 73.0},
 (1996, 'championship'): {'home_elo': 70.0, 'away_elo': 70.0},
 (1997, 'championship'): {'home_elo': 72.0, 'away_elo': 72.0},
 (1997, 'ligue_2'): {'home_elo': 72.0, 'away_elo': 72.0},
 (1998, 'championship'): {'home_elo': 74.0, 'away_elo': 74.0},
 (1998, 'ligue_2'): {'home_elo': 74.0, 'away_elo': 74.0},
 (1998, 'serie_b'): {'home_elo': 74.0, 'away_elo': 74.0},
 (1999, 'championship'): {'home_elo': 69.0, 'away_elo': 69.0},
 (1999, 'eerste_divisie'): {'home_elo': 69.0, 'away_elo': 69.0},
 (1999, 'serie_b'): {'home_elo': 69.0, '

In [39]:
for year_league in missing_elo_full_season_idx:
    
    year, league = year_league
    selection_idx = (scores_match_team_info_elo_df.season_year == year) & (scores_match_team_info_elo_df.league == league)

    scores_match_team_info_elo_df.loc[selection_idx, ['home_elo', 'away_elo']] = (scores_match_team_info_elo_df
        .query("(season_year == @year) and (league == @league)")
        [['home_elo', 'away_elo']]
        .fillna(median_values_for_full_season_missing_elo[year_league])
    )

The rest of the missing elo data will be filled in later on using ffill

### Fill in missing cards and Elo data with ffill

In [40]:
home_data = (scores_match_team_info_elo_df[['home_team', 'home_yellow', 'home_red', 'home_elo', 'match_round', 'season_year', 'league']]
                .rename(columns=lambda col_name: col_name[5:] if col_name[:4] == 'home' else col_name))
away_data = (scores_match_team_info_elo_df[['away_team', 'away_yellow', 'away_red', 'away_elo', 'match_round', 'season_year', 'league']]
                .rename(columns=lambda col_name: col_name[5:] if col_name[:4] == 'away' else col_name))

scores_data_long_format = (pd.concat([home_data, away_data])
                            .reset_index()
                            .assign(idx = lambda df_: np.arange(df_.shape[0]))
                            .set_index('idx'))

scores_data_long_new_features = (scores_data_long_format
    .sort_values(['league', 'season_year', 'team', 'match_round']).reset_index()
    .assign(yellow = lambda df_: df_.groupby(['team']).yellow.transform('ffill'),
        red = lambda df_: df_.groupby(['team']).red.transform('ffill'),
        elo = lambda df_: df_.groupby(['team']).elo.transform('ffill'))
    .set_index('idx').sort_index()
)

home_data_transformed = (scores_data_long_new_features[:home_data.shape[0]]
                            .set_index('match_id')
                            .drop(columns=['team', 'match_round',
                                'season_year', 'league'])
                            .rename(columns = lambda col: 'home_' + col))

away_data_transformed = (scores_data_long_new_features[home_data.shape[0]:]
                            .set_index('match_id')
                            .drop(columns=['team', 'match_round',
                                'season_year', 'league'])
                            .rename(columns = lambda col: 'away_' + col))

scores_match_team_info_elo_df = (scores_match_team_info_elo_df
    .drop(columns = ['home_yellow', 'home_red', 'away_yellow', 'away_red', 'home_elo', 'away_elo'])
    .join(home_data_transformed)
    .join(away_data_transformed)
    .assign(home_yellow = lambda df_: df_.groupby(['season_year', 'league', 'home_team']).home_yellow.transform('bfill'),
        home_red = lambda df_: df_.groupby(['season_year', 'league', 'home_team']).home_red.transform('bfill'),
        away_yellow = lambda df_: df_.groupby(['season_year', 'league', 'away_team']).away_yellow.transform('bfill'),
        away_red = lambda df_: df_.groupby(['season_year', 'league', 'away_team']).away_red.transform('bfill'))
    .assign(home_yellow = lambda df_: df_.groupby(['league', 'home_team']).home_yellow.transform('bfill'),
        home_red = lambda df_: df_.groupby(['league', 'home_team']).home_red.transform('bfill'),
        away_yellow = lambda df_: df_.groupby(['league', 'away_team']).away_yellow.transform('bfill'),
        away_red = lambda df_: df_.groupby(['league', 'away_team']).away_red.transform('bfill'))
    .drop(columns = ['date', 'referee'])
)

In [41]:
scores_match_team_info_elo_df.isna().sum()[scores_match_team_info_elo_df.isna().sum() > 0]

Series([], dtype: int64)

## Create new features

 - **Form**: points gained from the last 5 games, start from zero at the start of every season `home_form`, `away_form`
 
 - **Home/Away Form**: points gained from the last 3 (home matches for home team and away matches for away team), start from zero at the start of every season `home_team_home_form`, `away_team_away_form`
 
 - **Season Goals**: Cumulative sum of all season goals scored `home_total_goals`, `away_total_goals`
 
 - **Home/Away Season Goals**: Cumulative sum of all season goals scored in (home matches for home team and away matches for away team), `home_team_home_total_goals`, `away_team_away_total_goals`
 
 - **Discipline**: A made-up construction for yellow and red cards $(\text{Discipline for each game} = \text{Number of reds}+0.2(\text{Number of yellows}))$\
Then aggregate the discipline record as sum of last 5 games, start from zero at the start of every season `home_discipline`, `away_discipline`

> **Note**: For all these new features, the result has to be shifted by one so that only past information is included, i.e. no data leakage 

In [42]:
# Form, home and away, points gained from last 5 games, start from zero at start of every season
# Home/away form, points gained from last 3 home/away games, start from zero at start of every season
# Season goals, home and away, goals scored from the start of the season
# discipline, discipline record for last 5 games, n_reds + 0.2*n_yellows

In [43]:
new_features = ['home_form','away_form', 'home_total_goals', 'away_total_goals', 'home_discipline', 'away_discipline',
                    'home_team_home_form', 'home_team_home_total_goals', 'away_team_away_form', 'away_team_away_total_goals']

In [44]:
home_away_features_df = (scores_match_team_info_elo_df
    .assign(home_team_home_form = scores_match_team_info_elo_df.groupby(['home_team', 'season_year']).home_points.transform(lambda df: df.rolling(3, min_periods=1).sum().shift(1).fillna(0)),
        away_team_away_form = scores_match_team_info_elo_df.groupby(['away_team', 'season_year']).away_points.transform(lambda df: df.rolling(3, min_periods=1).sum().shift(1).fillna(0)),
        home_team_home_total_goals = scores_match_team_info_elo_df.groupby(['home_team', 'season_year']).home_goals.transform(lambda df: df.cumsum().shift(1).fillna(0)),
        away_team_away_total_goals = scores_match_team_info_elo_df.groupby(['home_team', 'season_year']).away_goals.transform(lambda df: df.cumsum().shift(1).fillna(0)))
)

(home_away_features_df.query("(home_team == 'Arsenal') and (season_year == 2020)"))

,home_team,away_team,score,link,season_year,match_round,league,home_goals,away_goals,result,...,home_yellow,home_red,home_elo,away_yellow,away_red,away_elo,home_team_home_form,away_team_away_form,home_team_home_total_goals,away_team_away_total_goals
match_id,,,,,,,,,,,,,,,,,,,,,
arsenal/burnley-fc/2020,Arsenal,Burnley,2-1,https://www.besoccer.com/match/arsenal/burnley...,2020,2,premier_league,2,1,home_win,...,2.0,0.0,91.0,1.0,0.0,72.0,0.0,0.0,0.0,0.0
arsenal/tottenham-hotspur-fc/2020,Arsenal,Tottenham Hotspur,2-2,https://www.besoccer.com/match/arsenal/tottenh...,2020,4,premier_league,2,2,draw,...,3.0,0.0,91.0,5.0,0.0,92.0,3.0,1.0,2.0,1.0
arsenal/aston-villa-fc/2020,Arsenal,Aston Villa,3-2,https://www.besoccer.com/match/arsenal/aston-v...,2020,6,premier_league,3,2,home_win,...,7.0,1.0,91.0,1.0,0.0,75.0,4.0,0.0,4.0,3.0
arsenal/afc-bournemouth/2020,Arsenal,AFC Bournemouth,1-0,https://www.besoccer.com/match/arsenal/afc-bou...,2020,8,premier_league,1,0,home_win,...,1.0,0.0,91.0,2.0,0.0,73.0,7.0,6.0,7.0,5.0
arsenal/crystal-palace-fc/2020,Arsenal,Crystal Palace,2-2,https://www.besoccer.com/match/arsenal/crystal...,2020,10,premier_league,2,2,draw,...,2.0,0.0,91.0,0.0,0.0,73.0,7.0,6.0,8.0,5.0
arsenal/wolverhampton/2020,Arsenal,Wolves,1-1,https://www.besoccer.com/match/arsenal/wolverh...,2020,11,premier_league,1,1,draw,...,0.0,0.0,91.0,2.0,0.0,72.0,7.0,5.0,10.0,7.0
arsenal/southampton-fc/2020,Arsenal,Southampton,2-2,https://www.besoccer.com/match/arsenal/southam...,2020,13,premier_league,2,2,draw,...,6.0,0.0,91.0,2.0,0.0,78.0,5.0,1.0,11.0,8.0
arsenal/brighton-amp-hov/2020,Arsenal,Brighton & Hove Albion,1-2,https://www.besoccer.com/match/arsenal/brighto...,2020,15,premier_league,1,2,away_win,...,3.0,0.0,91.0,1.0,0.0,69.0,3.0,0.0,13.0,10.0
arsenal/manchester-city-fc/2020,Arsenal,Man. City,0-3,https://www.besoccer.com/match/arsenal/manches...,2020,17,premier_league,0,3,away_win,...,1.0,0.0,90.0,4.0,0.0,96.0,2.0,4.0,14.0,12.0


In [45]:
home_data = (scores_match_team_info_elo_df[['home_team', 'home_goals', 'home_yellow', 'home_red', 'home_points', 'match_round', 'season_year', 'league']]
                .rename(columns=lambda col_name: col_name[5:] if col_name[:4] == 'home' else col_name))
away_data = (scores_match_team_info_elo_df[['away_team', 'away_goals', 'away_yellow', 'away_red', 'away_points', 'match_round', 'season_year', 'league']]
                .rename(columns=lambda col_name: col_name[5:] if col_name[:4] == 'away' else col_name))

scores_data_long_format = (pd.concat([home_data, away_data])
                            .reset_index()
                            .assign(idx = lambda df_: np.arange(df_.shape[0]))
                            .set_index('idx'))

scores_data_long_new_features = (scores_data_long_format
    .sort_values(['league', 'season_year', 'team', 'match_round']).reset_index()
    .assign(form = lambda df_: df_.groupby(['team', 'season_year']).points.transform(lambda df: df.rolling(5, min_periods=1).sum().shift(1).fillna(0)),
        total_goals = lambda df_: df_.groupby(['team', 'season_year']).goals.transform(lambda df: df.cumsum().shift(1).fillna(0)))
    .assign(cards_temp = lambda df_: df_.red.add(df_.yellow.mul(0.2)))
    .assign(discipline = lambda df_: df_.groupby(['team', 'season_year']).cards_temp.transform(lambda df: df.rolling(5, min_periods=1).sum().shift(1).fillna(0)))
    .set_index('idx').sort_index()
)

(scores_data_long_new_features
    .query("(team == 'Arsenal') and (season_year == 2020)")
    .sort_values('match_round')
)

,match_id,team,goals,yellow,red,points,match_round,season_year,league,form,total_goals,cards_temp,discipline
idx,,,,,,,,,,,,,
134355,newcastle-united-fc/arsenal/2020,Arsenal,1,3.0,0.0,3,1,2020,premier_league,0.0,0.0,0.6,0.0
11738,arsenal/burnley-fc/2020,Arsenal,2,2.0,0.0,3,2,2020,premier_league,3.0,1.0,0.4,0.6
134373,liverpool/arsenal/2020,Arsenal,1,1.0,0.0,0,3,2020,premier_league,6.0,3.0,0.2,1.0
11767,arsenal/tottenham-hotspur-fc/2020,Arsenal,2,3.0,0.0,1,4,2020,premier_league,6.0,4.0,0.6,1.2
134395,watford-fc/arsenal/2020,Arsenal,2,3.0,0.0,1,5,2020,premier_league,7.0,6.0,0.6,1.8
11786,arsenal/aston-villa-fc/2020,Arsenal,3,7.0,1.0,3,6,2020,premier_league,8.0,8.0,2.4,2.4
134416,manchester-united-fc/arsenal/2020,Arsenal,1,2.0,0.0,1,7,2020,premier_league,8.0,11.0,0.4,4.2
11804,arsenal/afc-bournemouth/2020,Arsenal,1,1.0,0.0,3,8,2020,premier_league,6.0,12.0,0.2,4.2
134436,sheffield-united/arsenal/2020,Arsenal,0,4.0,0.0,0,9,2020,premier_league,9.0,13.0,0.8,4.2


In [46]:
home_data_transformed = (scores_data_long_new_features[:home_data.shape[0]]
                            .set_index('match_id')
                            .drop(columns=['team', 'goals', 'yellow', 'red', 'points', 'match_round',
                                'season_year', 'league', 'cards_temp'])
                            .rename(columns = lambda col: 'home_' + col))

away_data_transformed = (scores_data_long_new_features[home_data.shape[0]:]
                            .set_index('match_id')
                            .drop(columns=['team', 'goals', 'yellow', 'red', 'points', 'match_round',
                                'season_year', 'league', 'cards_temp'])
                            .rename(columns = lambda col: 'away_' + col))

home_data_transformed.shape, away_data_transformed.shape


((122619, 3), (122619, 3))

In [47]:
transformed_df = (home_away_features_df
    .join(home_data_transformed)
    .join(away_data_transformed)
)

transformed_df.sample(5)

,home_team,away_team,score,link,season_year,match_round,league,home_goals,away_goals,result,...,home_team_home_form,away_team_away_form,home_team_home_total_goals,away_team_away_total_goals,home_form,home_total_goals,home_discipline,away_form,away_total_goals,away_discipline
match_id,,,,,,,,,,,,,,,,,,,,,
venezia/catania/2003,Venezia,Catania,2-1,https://www.besoccer.com/match/venezia/catania...,2003,16,serie_b,2,1,home_win,...,3.0,0.0,5.0,8.0,7.0,14.0,2.4,6.0,16.0,3.4
paris-saint-germain-fc/bordeaux/2002,PSG,Girondins Bordeaux,1-0,https://www.besoccer.com/match/paris-saint-ger...,2002,25,ligue_1,1,0,home_win,...,4.0,4.0,16.0,7.0,8.0,31.0,2.6,9.0,23.0,1.0
athletic-bilbao/espanyol/2007,Athletic,Espanyol,2-1,https://www.besoccer.com/match/athletic-bilbao...,2007,30,primera_division,2,1,home_win,...,3.0,4.0,12.0,22.0,4.0,32.0,2.4,6.0,32.0,2.0
bordeaux/troyes/2003,Girondins Bordeaux,Troyes,1-0,https://www.besoccer.com/match/bordeaux/troyes...,2003,6,ligue_1,1,0,home_win,...,1.0,2.0,1.0,2.0,8.0,5.0,4.6,5.0,2.0,2.6
dijon-fco/lillestrom/2012,Dijon FCO,Lille,0-2,https://www.besoccer.com/match/dijon-fco/lille...,2012,33,ligue_1,0,2,away_win,...,3.0,3.0,22.0,21.0,7.0,37.0,2.2,12.0,60.0,2.4


In [48]:
transformed_df.isna().sum()[transformed_df.isna().sum() > 0]

Series([], dtype: int64)

In [49]:
transformed_df.columns

Index(['home_team', 'away_team', 'score', 'link', 'season_year', 'match_round',
       'league', 'home_goals', 'away_goals', 'result', 'home_points',
       'away_points', 'capacity', 'missing_elo', 'missing_cards',
       'home_yellow', 'home_red', 'home_elo', 'away_yellow', 'away_red',
       'away_elo', 'home_team_home_form', 'away_team_away_form',
       'home_team_home_total_goals', 'away_team_away_total_goals', 'home_form',
       'home_total_goals', 'home_discipline', 'away_form', 'away_total_goals',
       'away_discipline'],
      dtype='object')

In [50]:
transformed_df.head()[['result', 'home_points', 'away_points']]

,result,home_points,away_points
match_id,,,
charlton-athletic-fc/derby-county-fc/1990,draw,1,1
tottenham-hotspur-fc/luton-town-fc/1990,home_win,3,0
southampton-fc/millwall-fc/1990,away_win,0,3
sheffield-wednesday-fc/norwich-city-fc/1990,away_win,0,3
queens-park-rangers-fc/crystal-palace-fc/1990,home_win,3,0


## Create a new cleaned dataset

In [51]:
cleaned_dataset_df = (transformed_df
    .reset_index()
    .drop(columns = ['match_id', 'home_team', 'away_team', 'score', 'link', 'league',
        'home_goals', 'away_goals', 'home_points', 'away_points', 'home_yellow', 'home_red',
        'away_yellow', 'away_red']))

cleaned_dataset_df.sample(5)

,season_year,match_round,result,capacity,missing_elo,missing_cards,home_elo,away_elo,home_team_home_form,away_team_away_form,home_team_home_total_goals,away_team_away_total_goals,home_form,home_total_goals,home_discipline,away_form,away_total_goals,away_discipline
100277,2017,15,home_win,20087.0,False,False,69.0,62.0,6.0,3.0,5.0,9.0,8.0,8.0,2.2,5.0,16.0,2.4
85129,2009,17,draw,7500.0,False,False,52.0,56.0,5.0,3.0,13.0,10.0,7.0,16.0,2.0,9.0,16.0,4.2
87486,2014,22,draw,37242.0,False,False,67.0,59.0,9.0,1.0,17.0,7.0,10.0,33.0,4.0,1.0,24.0,2.0
48532,1991,23,home_win,49780.0,False,False,79.0,79.0,5.0,3.0,23.0,11.0,6.0,34.0,1.0,6.0,27.0,2.0
89000,2017,33,away_win,17875.0,False,False,54.0,54.0,7.0,0.0,21.0,13.0,9.0,36.0,4.6,6.0,29.0,4.2


In [54]:
cleaned_dataset_df.shape

(122619, 18)

In [52]:
cleaned_dataset_df.to_csv('cleaned_dataset.csv', index=False)

In [55]:
cleaned_dataset_copy_df = pd.read_csv('cleaned_dataset.csv')
cleaned_dataset_copy_df.head()

,season_year,match_round,result,capacity,missing_elo,missing_cards,home_elo,away_elo,home_team_home_form,away_team_away_form,home_team_home_total_goals,away_team_away_total_goals,home_form,home_total_goals,home_discipline,away_form,away_total_goals,away_discipline
0,1990,1,draw,27111.0,False,False,72.0,74.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1990,1,home_win,62062.0,False,False,85.0,83.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1990,1,away_win,32689.0,False,False,81.0,68.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1990,1,away_win,33143.0,False,False,79.0,81.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1990,1,home_win,33143.0,False,False,83.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
